In [ ]:
!pip install music21
!pip install midi2audio
import numpy as np
import music21
music21.configure.run()
import random


______________________________________________________________________________ 
  
Welcome the music21 Configuration Assistant. You will be guided through a 
number of questions to install and setup music21. Simply pressing return at a 
prompt will select a default, if available. 

You may run this configuration again at a later time by running 
music21/configure.py. 
  
______________________________________________________________________________ 
  
Would you like to install music21 in the normal place for Python packages 
(i.e., site-packages)? Enter Yes or No (default is Yes): Yes

______________________________________________________________________________ 
  
The BSD/LGPL licensed music21 software is distributed with a corpus of encoded 
compositions which are distributed with the permission of the encoders (and, 
where needed, the composers or arrangers) and where permitted under United 
States copyright law. Some encodings included in the corpus may not be used 
for commerc

In [ ]:
def translate_notation(swaras):
  hindi = ['S',r'r','R','g','G','M','m','P','d','D','n','N']
  english = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
  trans =  dict(zip(hindi, english))
  note = []
  for s in swaras:
    if s!=',':
      octave = s[1]
      note.append(trans[s[0]]+octave)
    else:
      note.append(s)
  return note

def create_noteset():
  noteset = []
  hindi = ['S',r'r','R','g','G','M','m','P','d','D','n','N']
  for ix in range(3,6):
    for n in hindi:
      nt = n + str(ix)
      noteset.append(nt)
  noteset.append(',')
  return noteset

In [ ]:
# Upload bhairavi.dat (or an .dat file)

filename = '/content/bhairavi.dat'
unotes = translate_notation(create_noteset())
piecewise_transition_probs = []
p_transition_probs = np.zeros((len(unotes),len(unotes)))

for line in open(filename):
  if len(line) > 0:
    line = line.strip()
    # Counting note transitions
    prev_note = ',' # start with a blank note
    print(line)
    notes = translate_notation(line.split(' '))
    for p_note in notes:
      if len(p_note) > 0:
        try:
          toInd = unotes.index(p_note)
          fromInd = unotes.index(prev_note)
          p_transition_probs[fromInd,toInd] += 1
        except:
          print('\nNote incompatible with raga found:'+p_note+", ignoring note")
        if p_note == ',': pass # retain previous note on sustain
        else: 
          prev_note = p_note # update previous note to current note if it was not sustain


S4 r4 g4 M4 P4 d4 n4 S5 , n4 d4 P4 M4 g4 r4 S4
M4 , g4 M4 g4 , r4 S4 , d3 n3 S4 g4 r4 , S4 , ,
n3 , , S4 n3 , d3 n3 S4
S4 g4 r4 n3 S4 , , d4 P4 , d4 S4 n4
d4 P4 , M4 g4 M4 , g4 M4 g4 , r4 S4 , r4 S4
d4 , d4 , M4 d4 n4 S5 S5 S5 , n4 n4 , S5 ,
S5 g5 r5 S5 r5 , S5 n4 S5 n4 d4 P4 S5 , d4 d4 P4 ,
P4 d4 n4 S5 P4 n4 d4 P4 S5 S5 d4 , P4 , P4 P4
P4 d4 n4 S5 P4 n4 d4 P4 g4 d4 d4 P4 , d4 S5 n4
d4 P4 M4 g4 , P4 M4 M4 d4 P4 M4 g4 M4 g4 g4 , r4 S4
g4 S4 r4 S4 , g4 M4 P4 , d4 M4 d4 n4 S5 , r5 S5 d4 P4 g4 M4 r4 S4 ,
r4 g4 r4 g4 S5 r4 S4 , g4 M4 P4 , P4 d4 P4 , P4 d4 n4 d4 M4 ,
d4 P4 g4 M4 , P4 M4 g4 M4 r4 r4 S4 , d3 S4 , S4 g4 M4 P4 , P4 d4 P4
g4 r4 S4 , S4 g4 r4 M4 g4 , S4 g4 P4 M4 g4 S4 r4 S4 , n3 r4 n3 d3 n3 d3 P3 P3 d3 n3 d3 S4
P4 P4 d4 M4 P4 g4 M4 P4 , d4 S5 n4 d4 P4 M4 P4
S4 g4 , M4 P4 M4 r4 g4 r4 S4 ,
S4 g4 , M4 P4 M4 r4 S4 r4 S4 ,
g4 M4 d4 n4 S5 , r5 g4 r5 S5
S5 r5 G5 r5 S5 r5 S5 d4 n4 d4 P4
n4 r5 G5 r5 S5 r5 S5 d4 n4 d4 P4
d4 d4 P4 n4 d4 P4 g4 M4 g4 r4 , S4
P4 M4 g4 r4 S4 , M4 d4 P4 M4 g4 r4 

In [ ]:
import pickle

with open("bhairavi.aut", "wb") as fp:   #Pickling
  pickle.dump(p_transition_probs, fp)

# Download this file to use in the Simulation GUI

In [ ]:
# convert frequencies to probabilities
ps = np.sum(p_transition_probs,1)
ps[np.where(ps==0)] = .1
p_transition_probs = np.transpose(np.transpose(p_transition_probs)/ps) 
piecewise_transition_probs.append(p_transition_probs)

# transition probabilities matrix between the unique notes in the piece
transition_probs = np.zeros((len(unotes),len(unotes)))
# computing generic transition_probs from piecewise_transition_probs
for transition_prob in piecewise_transition_probs:
    transition_probs += transition_prob


In [ ]:
# Simulation

note_space_duration = 0.75
num_beat_cycle = 25
alankar_density = 2
centre = int((len(unotes)-1)/3) 
prev_swar_ind = centre  # Initialize the automata with the tonic in the middle octave
taal = 16    # Number of beats in a beat cycle. Choosing 'teental'
part = music21.stream.Part(id='flute')
part.append(music21.instrument.Flute())
prev_note = music21.note.Note(unotes[prev_swar_ind])
backup_swar_ind = prev_swar_ind
yaman = ['S3','R3','G3','m3','P3','D3','N3','S4','R4','G4','m4','P4','D4','N4','S5','R5','G5','m5','P5','D5','N5',',']
bhairavi = ['S3','r3','g3','M3','P3','d3','n3','S4','r4','g4','M4','P4','d4','n4','S5','r5','g5','M5','P5','d5','n5',',']
bhupali = ['S3','R3','G3','P3','D3','S4','R4','G4','P4','D4','S5','R5','G5','P5','D5',',']
R1_notes = translate_notation(bhupali)

In [ ]:
for cycle in range(num_beat_cycle):
  notes_list = []
  measure = music21.stream.Measure(number=1)
  if cycle == 0:
    note = music21.note.Note(unotes[int((len(unotes)-1)/3)])
    note.duration.quarterLength = note_space_duration
    print(note)
    measure.append(note)
  else:
    for taal_num in range(taal):
      while np.sum(transition_probs[prev_swar_ind,:]) < 1.0:
        print('f')
        if prev_swar_ind > centre:
          prev_swar_ind -= 1
        else:
          prev_swar_ind += 1
      swar_ind = np.random.choice(np.arange(len(unotes)),p=transition_probs[prev_swar_ind,:])

      # Random seed for the alankar
      seed = random.uniform(-1, 1)
      if seed < 0 and taal_num != 0:
        notes_list[-1].duration.quarterLength = note_space_duration/alankar_density

        for ix in np.arange(note_space_duration/alankar_density,note_space_duration,note_space_duration/alankar_density):
          print("aaaaaa")
          if unotes[swar_ind] == ',':
            notes_list[-1].duration.quarterLength += note_space_duration/alankar_density
          else:
            # insert a note with duration alankar_density
            this_note = music21.note.Note(unotes[swar_ind])
            this_note.duration.quarterLength = note_space_duration/alankar_density
            notes_list.append(this_note)

            # update previous swar to fill in next note in alankar
            prev_swar_ind = swar_ind
            prev_note = this_note

            while unotes[swar_ind] not in R1_notes:
              if swar_ind > centre:
                swar_ind -= 1
              else:
                swar_ind += 1
            alankar_ind = R1_notes.index(unotes[swar_ind])
            # get index for next note in alankar
            if seed < 0.5:
              alankar_ind += 1
            else:
              alankar_ind -= 1

            if alankar_ind > len(R1_notes): 
              alankar_ind -= 2
            if alankar_ind < 0: 
              alankar_ind += 2
            
            swar_ind = unotes.index(R1_notes[alankar_ind])
      
      else:
        if unotes[swar_ind] == ',' and taal_num==0:
          # print("Wrong starting note!")
          while unotes[swar_ind] == ',':
              swar_ind = np.random.choice(np.arange(len(unotes)),p=transition_probs[backup_swar_ind,:])

      # extend note duration if current index denotes a ','
      if unotes[swar_ind] == ',':
        notes_list[-1].duration.quarterLength += note_space_duration
      else:
        # create a note object for one quarterLength
        this_note = music21.note.Note(unotes[swar_ind])
        this_note.duration.quarterLength = note_space_duration

        # add it to the list
        notes_list.append(this_note)
        # update note and swaram index for next iteration
        backup_swar_ind = prev_swar_ind
        prev_swar_ind = swar_ind
        prev_note = this_note

  for note in notes_list:
    print(note)
    measure.append(note)

  # add measure to part
  part.append(measure)

<music21.note.Note C>
aaaaaa
f
aaaaaa
f
aaaaaa
f
aaaaaa
aaaaaa
aaaaaa
aaaaaa
aaaaaa
<music21.note.Note G>
<music21.note.Note G#>
<music21.note.Note G>
<music21.note.Note G#>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note G#>
<music21.note.Note A>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note C>
<music21.note.Note C#>
<music21.note.Note D>
<music21.note.Note D#>
<music21.note.Note E>
<music21.note.Note C>
<music21.note.Note D>
<music21.note.Note D#>
<music21.note.Note E>
<music21.note.Note C>
<music21.note.Note D#>
<music21.note.Note E>
<music21.note.Note C#>
aaaaaa
aaaaaa
aaaaaa
aaaaaa
aaaaaa
f
aaaaaa
f
f
f
aaaaaa
f
aaaaaa
f
<music21.note.Note C>
<music21.note.Note A#>
<music21.note.Note C>
<music21.note.Note C>
<music21.note.Note A#>
<music21.note.Note C>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note A#>
<music21.note.Note G>
<music21.note.Note A>

In [ ]:
# create an empty score
simulation = music21.stream.Score()

# add the part to it
simulation.append(part)
fp = simulation.write('midi', fp='/content/bhupali.mid')



In [ ]:
# Play audio
from IPython.display import Audio
!apt install fluidsynth
!find / -name *.sf2
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!fluidsynth -ni font.sf2 bhupali.mid -F bhupali.wav -r 44100


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 30 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni